In [1]:
from IPython.display import display, HTML, Image, clear_output
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

<center><span style="color:blue;font-family:helvetica; font-size:3.5rem; font-weight:700;">Reading edf files in Visbrain Sleep</span></center>


# Objective
The objective of this notebook is to give an outline of how edf files are read in Visbrain. 

# Issues with Visbrain Sleep
Currently I have found three main issues with Visbrain Sleep when reading .edf files from Physionet:
- Visbrain Sleep cannot handle multiple sampling frequencies. The Physionet files have two different sampling frequencies:
 - 100 Hz for EEG Fpz-Cz, EEG Ps_Oz, and EOG horizontal
 - 1 Hz for Rep oro-nasal, EMG submental, Temp rectal, and Event marker
- above issue can be solved by using the MNE software to read the edf files. MNE can handle multiple sampling frequencies. However, there is an interface issue between MNE and Visbrain Sleep. When the values of a channel are recorded in $\mu V$, MNE converts the values to $V$, while Visbrain Sleep does not convert these values. This issue is solved in Visbrain  in the the function `mne_switch` of the program `mneio.py` (for details, see below)
- in the header of the edf files, the following is specified:
 - the number of data records (`n_records`)
 - record length in seconds (`record_length`)
 - sample frequency per data record or samples per record for each channel (`n_samples_per_record`)  
 
 for the Physionet Sleep files, the record length in seconds is typically 30. The sampling frequency per record is typically 3000 for the first three channels and 30 for the others. This means that the actual sampling frequency is 100 Hz and 1 Hz respectively. However, when determining the time axis in Visbrain Sleep, the sample frequency per data record (3000) is used, while the actual sample frequency of 100 Hz should be used. It looks like the program is only tested for situations where the number of data records per second is 1. This issue is fixed in the program `read_sleep.py` (for details see below)  
- Visbrain Sleep has a very rudimentary module for reading hypnogram files and therefore cannot read the hypnogram files which are provided by Physionet. This means that the hypnogram files as provided by Physionet have to be converted to a form which is readable for Visbrain Sleep. This issue is dealt with in a separate notebook  

# The fixes
Two fixes are required to make the Visbrain Sleep software work correctly for the Physionet files:
- the variable `sf` has to be set to the sampling frequency instead of the number of samples per data record
- when using the MNE software to read the data files, the units of measurement for the channels measured in Volts has to be changed back from $V$ to $\mu V$  

In order to fix those two issues, the following has been done:
- in the Visbrain program `io/read_sleep.py`, line 305 has been commented out and line 306 has been added:  
```Python
305    #freqs = np.unique(edf.hdr['n_samples_per_record'])
306    freqs = np.unique(edf.hdr['n_samples_per_record'])/edf.hdr['record_length']
307    sf = freqs.max()```
This sets the variable `sf` to the sampling frequency in Hz and consequently, the time axis is computed correctly
- in the Visbrain program `io/mneio.py`, line 68 had been added to convert the values of the channels back from $v$ to $\mu V$:  
```Python
    data = (data.T/raw._raw_extras[0]['units']).T   # kdl added this line to change values  
                                                    # in Volt (MNE) to microVolt (Visbrain) ```


# The output
After fixing the first three issues, the plot of the data in Visbrain Sleep (when using MNE for reading all the channels) looks very similar to the plots in [PhysioBank ATM](https://physionet.org/cgi-bin/atm/ATM) as shown in the figures below where the values are plotted for the first 10 seconds. Only the first three channels are shown in the figures below. The other channels are more difficult to compare due to the fact that the y-axis for the plots is differently set by the two different software tools used for the plotting.    
<img src=figs/fig_6.png>  

<img src=figs/fig_7.png>

# EDF format
The sleep data from Physionet as well as the database on apnia sleep studies is using edf files. Reason why the emphasis on reading and processing files is on the edf format.

Initially there was $EDF$ and then $EDF^+$. I believe that $EDF^+$  can contain interrupted signals.

The specification can be found [here](http://edfplus.info/index.html).  

# Test program
To understand how Visbrain reads edf files, a program is debugged which reads an edf file and then plots the results. This program is:  
```Python
# Read edf file from library
from visbrain import Sleep

Sleep("/users/kees/sleepdynamics/sleep-edfx/SC4001E0-PSG.edf").show()```  

The `SC4001E0-PSG.edf` file is the first polysomnography file in the Physionet sleep database.  

`SC4001E0-PSG.edf` is the first file in the Physionet database. The plots generated by the Visbrain software can be visually compared with the plots from [PhysioBank ATM](https://physionet.org/cgi-bin/atm/ATM).  

The program is initially run by pressing "Cancel" when the popup for a hypnogram file is shown.  


# `from visbrain import Sleep`
This is the first line in the test program discussed in the previous section. Below an outline of the processes triggerd by this line:

In sitepackages in the conda environment `sleepdynamics`, there is a `visbrain.egg-link` with the following lines:
```txt
/Users/kees/SleepDynamics/visbrain
.```  

This makes the link to the `visbrain` folder in `SleepDynamics` where the visbrain programs are stored.  

The following is done when `from visbrain import Sleep` is executed:
- `visbrain/__init__.py` is loaded. This import the following modules where links are made to the appropriate program:
 - `Brain`  
 visualize EEG/MEG/Intracranial data and connectivity in a standard
  MNI 3D brain
 - `Colorbar`  
 a colorbar editor
 - `Figure`  
 figure-layout for high-quality publication-like figures.
 - `Sleep`  
 visualize polysomnographic data and hypnogram edition
 - `Topo`  
 topographic representations
 - `Signal`  
 data mining module for signal inspection
- only `Sleep` is really imported form `visbrain/visbrain/sleep/sleep.py`. When this happens, the following is done:  
```Python
import vispy.scene.cameras as viscam
from .interface import UiInit, UiElements
from .visuals import Visuals
from ..pyqt_module import PyQtModule
from ..utils import (FixedCam, color2vb, MouseEventControl)
from ..io import ReadSleepData
from ..config import PROFILER
class Sleep(PyQtModule, ReadSleepData, UiInit, Visuals, UiElements,
            MouseEventControl):
    """Visualize and edit sleep data.```
- the `Sleep` module is used for:
 - load and visualise polysomnographic data and spectrogram
 - load, edit and save hypnogram from the interface
 - perform automatic events detection
 - further signal processing tools (de-mean, de-trend and filtering)
 - topographic data visualisation  
 

# `Sleep(data=dfile).show()`
`sleep(data = dfile).show()` is the second line in the test program.  

The following parameters can be used for `Sleep.show()` command:
- `data`: path and name of the polysomnographic data
- `hypno`: path and name of the hypnogram data
- `config_file`: path and name of the configuration file (.txt)= default - None
- `annotations`: path to the annotation file (.txt, .csv). Alternatively, you can pass an annotation instance of MNE or simpy an (N,) array describing the onset
- `channels`: list of channel names. The length of this list must be n_channels; default = None
- `sf`: the sampling frequency of raw data; default = None
- `downsample`: default = 100
- `axis`: specify if each axis have to contains it own axis
- `href`: list of sleep stages; Used for display in GII; default = `['art', 'wake', 'rem', 'n1', 'n2', 'n3']`  


This notebook only deals with reading an edf file using Visbrain Sleep. How to read an edf file into Visbrain Sleep using the MNE software as well as how to read a corresponding hypnogram file are discussed in separate notebooks. 

## Outline of program `sleep.py`
The class`Sleep` which is called in the test program is a class defined in `sleep.py`. The parts in this class relevant to reading the data are:
```Python
"""Top Level Sleep class"""
# import modules
...
class Sleep(PyQtModule, ReadSleepData, UiInit, Visuals, UiElements,
            MouseEventControl):
    def __init__(self, data=None, hypno=None, config_file=None,
                 annotations=None, channels=None, sf=None, downsample=100.,
                 axis=True, href=['art', 'wake', 'rem', 'n1', 'n2', 'n3'],
                 preload=True, use_mne=False, kwargs_mne={}, verbose=None):
...
      ReadSleepData.__init__(self, data, channels, sf, hypno, href, preload,
                               use_mne, downsample, kwargs_mne,
                               annotations)```

### `ReadSleepData`
`ReadSleepData` is a class defined in `read_sleep.py`
```Python
class ReadSleepData(object):
    """Main class for reading sleep data."""

    def __init__(self, data, channels, sf, hypno, href, preload, use_mne,
                 downsample, kwargs_mne, annotations):
        # dialog window if data is none
        ...
        if use_mne:
            ...
            args = mne_switch(file, ext, downsample, **kwargs_mne)
        else:
            ...
            args = sleep_switch(file, ext, downsample)
        ```
        
`sleep_switch` is a function defined in `read_sleep.py`. In this function it is determined which read program in Visbrain is used depending on the extension of the filename. The relevant statements are: 
 ```Python
    if ext == '.vhdr':  # BrainVision
        return read_eeg(path, downsample)
    if ext == '.eeg':  # Elan
        return read_elan(path, downsample)
    elif ext in ['.edf', '.rec']:  # European Data Format
        return read_edf(path, downsample)
    elif ext == '.trc':  # Micromed
        return read_trc(path, downsample)
    else:  # None
        raise ValueError("*" + ext + " files are currently not supported.")```

In the situation that there is .edf file, the program used for reading the data is `read_edf`
- first two statements in `read_edf.py`:
 ```Python
    from ..utils.sleep.edf import Edf
    edf = Edf(path)```
 - `Edf` is a class in `visbrain/utils/sleep/edf.py`   
 
 So in `read_sleep.py` an instance is created of `Edf` which is in `edf.py`. 

### `edf.py` 
The main comment in `edf.py`:  
  ```Python
"""Module reads and writes header and data for EDF data.
Poor man's version of
https://github.com/breuderink/eegtools/blob/master/eegtools/io/edfplus.py
Values are slightly different from those computed by FieldTrip, however they
are identical to those computed by Biosig and EDFBrowser. The difference is due
to the calibration.
"""```

In class `Edf` in `edf.py`, the main steps are:
- some parameters are set:  
```Python
EDF_FORMAT = 'int16'  # by definition
edf_iinfo = iinfo(EDF_FORMAT)
DIGITAL_MAX = edf_iinfo.max
DIGITAL_MIN = -1 * edf_iinfo.max  # so that digital 0 = physical 0```
- the header data is read with method `_read_hdr(self)`
- the following methods are defined and available: 
 - `return_hdr(self)`
 - `_read_dat(self, i_chan, begsam, endsam)`
 - `return_dat(self, chan, begsam, endsam)`

#### Reading the header data
Reading the header data is triggered in the class `Edf` in `__init__` and uses the method `_read_hdr(self)`:
```Python
def _read_hdr(self):
    
    with open(self.filename, 'rb') as f:

        hdr = {}
        assert f.tell() == 0
        assert f.read(8) == b'0       '

        # recording info
        hdr['subject_id'] = f.read(80).decode('utf-8').strip()
        hdr['recording_id'] = f.read(80).decode('utf-8').strip()

        # parse timestamp
        (day, month, year) = [int(x) for x in findall(
            '(\d+)', f.read(8).decode('utf-8'))]
        (hour, minute, sec) = [int(x) for x in findall(
            '(\d+)', f.read(8).decode('utf-8'))]
        hdr['start_time'] = datetime(year + 2000, month, day, hour, minute,
                                     sec)

        # misc
        hdr['header_n_bytes'] = int(f.read(8))
        f.seek(44, 1)  # reserved for EDF+
        hdr['n_records'] = int(f.read(8))
        hdr['record_length'] = float(f.read(8))  # in seconds
        nchannels = hdr['n_channels'] = int(f.read(4))

        # read channel info
        channels = range(hdr['n_channels'])
        hdr['label'] = [f.read(16).decode('utf-8').strip() for n in
                        channels]
        hdr['transducer'] = [f.read(80).decode('utf-8').strip()
                             for n in channels]
        hdr['physical_dim'] = [f.read(8).decode('utf-8').strip() for n in
                               channels]
        hdr['physical_min'] = asarray([float(f.read(8))
                                       for n in channels])
        hdr['physical_max'] = asarray([float(f.read(8))
                                       for n in channels])
        hdr['digital_min'] = asarray([float(f.read(8)) for n in channels])
        hdr['digital_max'] = asarray([float(f.read(8)) for n in channels])
        hdr['prefiltering'] = [f.read(80).decode('utf-8').strip()
                               for n in channels]
        hdr['n_samples_per_record'] = [int(f.read(8)) for n in channels]
        f.seek(32 * nchannels, 1)  # reserved

        assert f.tell() == hdr['header_n_bytes']

        self.hdr = hdr```  
        

The header data is read as follows:
- `hdr = {}`: an empty directory is created for the header info
- the edf file is open as `f` with mode: `read, binary` 
- `hdr['subject_id']`: subject id which is 80 bytes is read, decode from `utf-8` to `string` and then stripped
- `hdr['recording_id]`: recording id which is 80 bytes is read, decode from `utf-8` to `string` and then stripped
- parse timestamp:
 - `f.read(8).decode('utf8')` gives the day/month/year as e.g. `24.09.89`
 - `findall('(d\+)` on above searches for groups of 1 or more digits; this would return here `[24 09 89]` 
 - the program uses a comprehension here. Note that this is not necessary as a list of the three numbers is already returned by the code above
 - the list is unpacked into `day, month, year`
 - exactly the same is done on the next 8 bytes which gives the `hour, minute, sec`
 - `hdr['start_time'] = datetime(year+2000, month, day, hour, minute, sec)`  
 note that the calculation of the year is too simple' we will get here 2089 instead of 1989; in example what is returned is `datetime.datetime(2089, 4, 24, 16, 13)`
- the next 8 bytes are read to give the number of bytes in the header; here `2048`
- `f.seek(44,1)`: the cursor is moved 44 forwards as these bytes are reserved for EDF+
- `hdr['n_records']` is read from the next 8 bytes; here `2650`
- `hdr['record_length']` (in seconds) is read from the next 8 bytes; here `30.0`
- `nchannels = hdr['n_channels']` is read from the next 4 bytes; here `7`
- read channel info:  
 - `channels = range(hdr['n_channels']`
 - the channel labels are read as:  
 `hdr['label'] = [f.read(16).decode('utf-8').strip() for n in channels]`
 - for each channel, the `transducer, physical_dim, physical_min, physical_max, digital_min, digital_max, prefiltering, 'n_samples_per_record` are all read in in a similar way for each channel (see above)
- `f.seek(32 * nchannels, 1)` the cursor is moved 32 bytes for each channel
- end of file is checked: `f.tell() == hdr['header_n-bytes']`
- `self.hdr = hdr`

 


In `read_sleep.py` the method `edf.return_hdr()` is used to return the data to `read_edf.py`. The code `return_hdr` is:
 ```Python
    def return_hdr(self):
        """Return the header for further use.

        Returns
        -------
        subj_id : str
            subject identification code
        start_time : datetime
            start time of the dataset
        s_freq : float
            sampling frequency
        chan_name : list of str
            list of all the channels
        n_samples : int
            number of samples in the dataset
        orig : dict
            additional information taken directly from the header
        """
        subj_id = self.hdr['subject_id']
        start_time = self.hdr['start_time']

        # Check that all channels have the same sampling rate
        # _assert_all_the_same(self.hdr['n_samples_per_record'])

        s_freq = (self.hdr['n_samples_per_record'][0] /
                  self.hdr['record_length'])

        chan_name = self.hdr['label']

        n_samples = (self.hdr['n_samples_per_record'][0] *
                     self.hdr['n_records'])

        return subj_id, start_time, s_freq, chan_name, n_samples, self.hdr```
        
Note that `self.hdr` already contains `subj_id`, `start_time`, and `chan_name` (list of channel names). `s_freq` and `n_samples` are added here. They are based on the information of the ***first channel***. This is later used to exclude channels which have a different number of samples. This is probably built in because of the fact that EEGTools, which is the basis for this edf read program, cannot handle multiple sample rates.
     

For the EEG file `SC4001E0-PSG.edf`, the values of the header are:  
```Text
subject_id :  X F X Female_33yr
recording_id :  Startdate 24-APR-1989 X X X
start_time :  2089-04-24 16:13:00
header_n_bytes :  2048
n_records :  2650
record_length :  30.0
n_channels :  7
label :  ['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 
             'Event marker']
transducer :  ['Ag-AgCl electrodes', 'Ag-AgCl electrodes', 'Ag-AgCl electrodes', 'Oral-nasal thermistors', 
                      'Ag-AgCl electrodes', 'Rectal  thermistor', 'Marker button']
physical_dim :  ['uV', 'uV', 'uV', '', 'uV', 'DegC', '']
physical_min :  [ -192.  -197. -1009. -2048.    -5.    34. -2047.]
physical_max :  [ 192.  196. 1009. 2047.    5.   40. 2048.]
digital_min :  [-2048. -2048. -2048. -2048. -2500. -2849. -2047.]
digital_max :  [2047. 2047. 2047. 2047. 2500. 2731. 2048.]
prefiltering :  ['HP:0.5Hz LP:100Hz [enhanced cassette BW]', 'HP:0.5Hz LP:100Hz [enhanced cassette BW]', 
                      'HP:0.5Hz LP:100Hz  [enhanced cassette BW]', 'HP:0.03Hz LP:0.9Hz', 
                      'HP:16Hz Rectification LP:0.7Hz', '', 'Hold during 2 seconds']
n_samples_per_record :  [3000, 3000, 3000, 30, 30, 30, 30]```

#### Processing the header data
After the header data is read in `edf.py` (class `Edf`), the header data is returned to `read_sleep.py` with the method `edf.return_hdr()`.  

This is a really messy program. In the method `return_hdr` a number of variables are computed and returned which are not used. It would be better to just return the header which is available as `edf.hdr` but also returned from `edf.return_hdr` and do all further processing in `read_sleep.py`.  

What happens in this program is that the sample frequencies of the various channels is compared and only the ones for which the sample frequency is equal to the maximum of these frequencies are selected for reading the data. This means that for the example outlined above, the ones selected are:
- `EEG Fpz-Cz`
- `EEG Pz-Oz`
- `EOG horizontal`  

The other four are not selected and therefore the data is not read. This is in contrast to what happens in MNE and Wonambi, where the other four types of data are read and incorporated into the plot.  

The code in `read_sleep.py` to eliminate channels is:
```Python
    # Return header informations
    _, start_time, sf, chan, n_samples, _ = edf.return_hdr()
    start_time = start_time.time()

    # Keep only data channels (e.g excludes marker chan)
    freqs = np.unique(edf.hdr['n_samples_per_record'])
    sf = freqs.max()

    if len(freqs) != 1:
        bad_chans = np.where(edf.hdr['n_samples_per_record'] < sf)
        chan = np.delete(chan, bad_chans)```

#### Reading the data
After selecting the channels for reading data, the `return_data` method of class in `edf.py` is accessed to read the data. The data is read into a 2 dimension numpy array where the channels rows and the samples are the columns. First the digital and physical minimum is extracted from the header data as a list and the digital and physical range as well as the gain are computed. The code is:  
```Python
        dig_min = hdr['digital_min']
        phys_min = hdr['physical_min']
        phys_range = hdr['physical_max'] - hdr['physical_min']
        dig_range = hdr['digital_max'] - hdr['digital_min']

        # assert all(phys_range > 0)
        # assert all(dig_range > 0)

        gain = phys_range / dig_range```
        
Then the data for the selected channels is read channel by channel with:
```Python
        dat = empty(shape=(len(chan), endsam - begsam), dtype='float64')

        for i, i_chan in enumerate(chan):
            d = self._read_dat(i, begsam, endsam).astype('float64')
            dat[i, :] = (d - dig_min[i]) * gain[i] + phys_min[i]

        return dat```
        
`(d - dig_min[i]) * gain[i] + phys_min[i]` is an adjustment of the values by calibration. At this stage I don't understand this. 

The method `_read_dat` looks pretty straightforward, but there is some manipulation concerning record positions which I like to work out.

## Conversion from digital values to physical values
Before the data is read, `gain` is computed for each channel as the ratio between `phys_range` and `dig_range`. For our example we have the following values:  
<img src=figs/fig_1.png>


At the time of reading the data,  an array `dat` is created as an empty array with dimensions $channels\times sample points$. The data is read per channel using the `_read_dat` method of `Edf`, but only for the EEG channels (channels with number of samples per data record = 3000). Initially the data per channel is read into the array `d`. These are values in the digital range. This is converted to the physical range and put into the array `dat`:  
```Python
        for i, i_chan in enumerate(chan):
            d = self._read_dat(i, begsam, endsam).astype('float64')
            dat[i, :] = (d - dig_min[i]) * gain[i] + phys_min[i]```
            
Note that unit of measurement of the data for the EEG channels $\mu V$.  

The data is then downsampled. There is a variable `downsample` which has as default 100. Initially in Visbrain `sf` is set to the number of samples per data record. This is not correct, `sf` should be set to the sample frequency in Hz. I have rectified this (see the section about fixes).  

The two variables `downsample` and `sf` are used to compute `dsf`, the downsampling factor:  
```Python
    if all([isinstance(k, (int, float)) for k in (downsample, sf)]):
        dsf = int(np.round(sf / downsample))
        downsample = float(sf / dsf)
        return dsf, downsample
    else:
        return 1, downsample```
        
The method `read_edf` then downsamples the data and returns the following values:
```Python
    return sf, downsample, dsf, data[:, ::dsf], chan, n, start_time, None```
    
When processing the Physionet files, the sampling frequency used for the EEG channels is 100 Hz. The default for the variable `downsample` is also 100. This means that the data is not downsampled as `dsf` is calculated as 1.

## Setting the time axis
The time axis is now determined as:
```Python
        time = np.arange(n)[::dsf] / sf```
        
Initially `sf` was set to the number of samples per data record. In the situation of the Physionet files, the number of samples per data record for the EEG channels is 3000. This results to an incorrect time axis. The correct way is to set `sf` to the sample frequency (for Physionet files the sample frequency = 100 Hz).  

The Visbrain program has been changed in order to correct this (see the section about fixes).